In [1]:
import sys

sys.path.insert(0,'/regelum-playground')

import numpy as np

from src.simulator_gz import Robot3Pi
from src.system_exp import ThreeWheelPushingObject
from src.environment import EnvironmentConfig, Task, TwistAction, PushingObject

In [2]:
## Possible Tasks (With either 15 or -15 rotation as starting point)
tasks = {}
tasks['red'] = Task('red',[Task.Transform(position=[0.4,0.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,0.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)
tasks['green'] = Task('green',[Task.Transform(position=[0.4,4.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,4.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)
tasks['blue'] = Task('blue',[Task.Transform(position=[0.4,8.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,8.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['yellow'] = Task('yellow',[Task.Transform(position=[0.4,-4.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-4.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['pink'] = Task('pink',[Task.Transform(position=[0.4,-8.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-8.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=0)
tasks['cyan'] = Task('cyan',[Task.Transform(position=[0.4,-12.0,0.05],euler_rotation=[0.0,0.0,15.0]),Task.Transform(position=[0.4,-12.0,0.05],euler_rotation=[0.0,0.0,-15.0])],mass=20)


actions = []

raw_action_space = 4.*np.array([[0.,0.],[0.5,0.1],[0.1,0.5],[0.3,0.3]]) ;

for e in raw_action_space:
    label = "" ;
    if e[0]+e[1] < 0.0001: label = "stop" ;
    elif e[0] == e[1]: label = "straight" ;
    elif e[0] >= e[1]: label = "right" ;
    elif e[0] <= e[1]: label = "left" ;
    actions.append(TwistAction(label, e))

In [3]:
system = ThreeWheelPushingObject()
env_config = EnvironmentConfig(observation_shape=[20,20,3],
                               tasks=tasks,
                               actions=actions,
                               robot_name='3pi_front_cam_robot',
                               vehicle_prefix='/vehicle',
                               world_name='/world/pushing_objects_world',
                               camera_topic='/vehicle/camera')

In [ ]:
sim_rob = Robot3Pi(system, 
                   state_init=np.zeros([20*20*3]),
                   time_final=30)

In [5]:
from src.objective import ThreeWheeledRobotCostWithSpot, PushingObjectRunningObjective
from regelum.model import ModelQuadLin


objective_function = ThreeWheeledRobotCostWithSpot(
    quadratic_model=ModelQuadLin(
        weights= [100, 100, 0, 0, 0],
        quad_matrix_type="diagonal",
        is_with_linear_terms=False),
        spot_gain=500,
        spot_x_center=-1.5,
        spot_y_center=-1.5,
        spot_std=0.2,
    )

objective_function = PushingObjectRunningObjective()

In [ ]:
po = PushingObject(sim_rob, 
                   running_objective=objective_function)

simulator: <src.simulator_gz.Robot3Pi object at 0x75829e51bfa0>
subscribed to Camera!
Subscribed to dynamic_pose/info!
Waiting for simulator... 0.24 sec
Scene received!
Model in scene empty_plane
Model in scene red
Model in scene green
Model in scene blue
Model in scene yellow
Model in scene pink
Model in scene cyan
Model in scene 3pi_front_cam_robot


In [13]:
po.switch_task(0)
observation, _ = po.reset()

In [15]:
import time

observation, _ = po.reset()

for _ in range(10):
    observation, reward, truncated, terminated, info = po.step(np.array([1, 0]))
    if np.logical_or(truncated, terminated):
        break
    
    time.sleep(0.2) 

po.step(np.array([0, 0]))

(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 16
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 20
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.42352942 0.85490197 30
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.617801 V 0.38039216 0.85490197 28
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.6666667 V 0.3529412 0.85490197 25
COND: Normal
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.0 V 0.42352942 0.85490197 0
COND: LOST
self.system.inputs: [[1. 0.]]
(20, 20, 3) VMINMAX H 0.0 0.0 S 0.0 0.0 V 0.42352942 0.85490197 0
COND: LOST
self.system.inputs: [[0. 0.]]


(
    array([[0.85490197, 0.85490197, 0.85490197, ..., 0.42352942, 0.42352942,
        0.42352942]], dtype=float32),
    1.0,
    True,
    False,
    {}
)